In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from time import time as timer
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import random_split, Dataset, DataLoader

In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/cifar-10/trainLabels.csv
/kaggle/input/cifar-10/sampleSubmission.csv
/kaggle/input/cifar-10/test.7z
/kaggle/input/cifar-10/train.7z


In [3]:
labels_csv = pd.read_csv("/kaggle/input/cifar-10/trainLabels.csv")
labels_csv

,id,label
0,1,frog
1,2,truck
2,3,truck
3,4,deer
4,5,automobile
...,...,...
49995,49996,bird
49996,49997,frog
49997,49998,truck
49998,49999,automobile


In [4]:
label_mapping = {label: idx for idx, label in enumerate(labels_csv['label'].unique())}

labels_csv.rename({"label": "label_txt"}, axis=1, inplace=True)
labels_csv['label'] = labels_csv['label_txt'].map(label_mapping)

print(label_mapping, labels_csv)

{'frog': 0, 'truck': 1, 'deer': 2, 'automobile': 3, 'bird': 4, 'horse': 5, 'ship': 6, 'cat': 7, 'dog': 8, 'airplane': 9}           id   label_txt  label
0          1        frog      0
1          2       truck      1
2          3       truck      1
3          4        deer      2
4          5  automobile      3
...      ...         ...    ...
49995  49996        bird      4
49996  49997        frog      0
49997  49998       truck      1
49998  49999  automobile      3
49999  50000  automobile      3

[50000 rows x 3 columns]


In [5]:
!pip install py7zr

import py7zr

archive_path = '/kaggle/input/cifar-10/train.7z'

extract_to = '/kaggle/working'

with py7zr.SevenZipFile(archive_path, mode='r') as z:
    z.extractall(path=extract_to)

print(f"Архив распакован в {extract_to}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 65.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.9/412.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.12 requires async-timeout<5.0.0,>=4.0.0; python_version 

In [6]:
train_dir = "/kaggle/working/train"
# test_dir = "/kaggle/working/test"

train_imgs = os.listdir(train_dir)

img1 = Image.open(train_dir + "/" + train_imgs[0])
img1.size

(32, 32)

In [7]:
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
])

In [8]:
class CustomDataset(Dataset):
    def __init__(self, images_dir, labels_csv, transform=None):
        self.transforms = transform
        self.imgs_dir = images_dir
        self.labels_csv = labels_csv

    def __len__(self):
        return len(self.labels_csv)

    def __getitem__(self, idx):
        img_name = str(self.labels_csv.iloc[idx, 0]) + ".png"
        label = self.labels_csv.loc[idx, "label"]

        img_path = os.path.join(self.imgs_dir, img_name)

        img = Image.open(img_path).convert("RGB")

        if self.transforms:
            img = self.transforms(img)

        return img, label

In [9]:
train_dataset = CustomDataset(
    images_dir = "/kaggle/working/train", labels_csv = labels_csv,
    transform = train_transform
)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [11]:
from tqdm import tqdm

import torch.nn as nn
import torch.nn.functional as F

class CNNModel(nn.Module):
    def __init__(self, num_classes=10):
        super(CNNModel, self).__init__()

        self.conv_block = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # [B, 32, 32, 32]
            nn.ReLU(),
            nn.MaxPool2d(2),  # [B, 32, 16, 16]

            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # [B, 64, 16, 16]
            nn.ReLU(),
            nn.MaxPool2d(2),  # [B, 64, 8, 8]

            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # [B, 128, 8, 8]
            nn.ReLU(),
            nn.MaxPool2d(2),  # [B, 128, 4, 4]
        )

        self.fc_block = nn.Sequential(
            nn.Flatten(),  # [B, 128*4*4]
            nn.Linear(128 * 4 * 4, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.conv_block(x)
        x = self.fc_block(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNModel(num_classes=10).to(device)

cr = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

corrects = 0
epochs = 10

for i in range(epochs):
    cost = 0
    for image, answer in tqdm(train_loader):
        image = image.to(device)
        answer = answer.to(device)
        # print(image, answer)

        optimizer.zero_grad()
        predictions = model(image)
        error = cr(predictions, answer)
        error.backward()
        optimizer.step()
        cost += error.item()

        if i == epochs - 1:
            corrects += (predictions.argmax(1) == answer).sum().item()

    print(f"Epoch {i+1}, Loss: {cost / len(train_loader):.4f}")

total_samples = len(train_loader.dataset)
print("Accuracy:", corrects, "/", total_samples, "=", corrects / total_samples)

 29%|██▉       | 459/1563 [00:11<00:28, 38.96it/s]


KeyboardInterrupt: 

In [13]:
from torchvision.models import resnet50, ResNet50_Weights
from tqdm import tqdm

weights = ResNet50_Weights.IMAGENET1K_V2
model = resnet50(weights=weights)

num_classes = 10
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)

'''
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True
'''

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

Epoch 1/10: 100%|██████████| 1563/1563 [01:28<00:00, 17.56it/s]


Epoch 1, Loss: 1.1071


Epoch 2/10: 100%|██████████| 1563/1563 [01:27<00:00, 17.84it/s]


Epoch 2, Loss: 0.7931


Epoch 3/10: 100%|██████████| 1563/1563 [01:27<00:00, 17.93it/s]


...

Epoch 8, Loss: 0.4820


Epoch 9/10: 100%|██████████| 1563/1563 [01:27<00:00, 17.85it/s]


Epoch 9, Loss: 0.4632


Epoch 10/10: 100%|██████████| 1563/1563 [01:26<00:00, 17.97it/s]

Epoch 10, Loss: 0.4301


In [ ]:
train_split = int(0.8 * len(train_dataset))
val_split = int(0.2 * len(train_dataset))
train_split, val_split

train_dataset, val_dataset = random_split(train_dataset, [train_split, val_split])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [14]:
import py7zr

archive_path = '/kaggle/input/cifar-10/test.7z'

extract_to = '/kaggle/working'

with py7zr.SevenZipFile(archive_path, mode='r') as z:
    z.extractall(path=extract_to)

print(f"Архив распакован в {extract_to}")

Архив распакован в /kaggle/working


In [15]:
class CustomTestDataset(Dataset):
    def __init__(self, images_dir, transform=None):
        self.images_dir = images_dir
        self.transform = transform
        self.img_files = sorted(os.listdir(images_dir))

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.images_dir, self.img_files[idx])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, self.img_files[idx]


test_dataset = CustomTestDataset("/kaggle/working/test", test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [16]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [17]:
all_preds = []
img_idxs = []

with torch.no_grad():
    for images, idxs in tqdm(test_loader):
        images = images.to(device)

        outputs = model(images)

        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        img_idxs.extend(idxs)

100%|██████████| 9375/9375 [03:44<00:00, 41.81it/s]


In [18]:
idxs_to_classes = {v: k for k, v in label_mapping.items()}

predicted_labels = [idxs_to_classes[pred] for pred in all_preds]

submission_df = pd.DataFrame({
    "id": [int(file.split(".")[0]) for file in img_idxs],
    "label": predicted_labels
})

submission_df = submission_df.sort_values(by="id")

submission_df.to_csv("submission.csv", index=False)

In [19]:
submission_df

,id,label
0,1,deer
111111,2,airplane
222222,3,automobile
233334,4,ship
244445,5,bird
...,...,...
222218,299996,truck
222219,299997,deer
222220,299998,frog
222221,299999,frog
